In [2]:
# =========================================
# YOLOv8 Colab 一體式完整範例（穩定版）
# =========================================

# ---------- 安裝套件（YOLOv8 不需要 TensorFlow） ----------
%pip install -q ultralytics opencv-python numpy==1.26.4

# ---------- 匯入模組 ----------
import os
import cv2
import numpy as np
from ultralytics import YOLO
from google.colab import drive
from IPython.display import HTML, display

# ---------- 掛載 Google Drive ----------
drive.mount('/content/drive', force_remount=True)

# ---------- 路徑設定 ----------
PROJECT_DIR = "/content/drive/MyDrive/20251226AI/yolo_project"
DATA_DIR = "/content/drive/MyDrive/20251226AI/yolo_data"

# ---------- 建立資料夾結構 ----------
for p in [
    PROJECT_DIR,
    f"{DATA_DIR}/images/train",
    f"{DATA_DIR}/images/val",
    f"{DATA_DIR}/labels/train",
    f"{DATA_DIR}/labels/val",
]:
    os.makedirs(p, exist_ok=True)

print("✅ 資料夾結構建立完成")

# ---------- 建立正確的 data.yaml ----------
yaml_path = f"{DATA_DIR}/data.yaml"
with open(yaml_path, "w") as f:
    f.write(f"""
path: {DATA_DIR}

train: images/train
val: images/val

nc: 2
names:
  - person
  - car
""")

print(f"✅ data.yaml 已寫入: {yaml_path}")

# ---------- YOLOv8 訓練 ----------
model = YOLO("yolov8n.pt")

model.train(
    data=yaml_path,
    epochs=1,          # 測試用，正式可加大
    imgsz=640,
    batch=16,
    project=PROJECT_DIR,
    name="yolo_finetune",
    verbose=True
)

# ---------- 載入訓練權重（若不存在則用預訓練） ----------
trained_path = f"{PROJECT_DIR}/yolo_finetune/weights/best.pt"

if os.path.exists(trained_path):
    model = YOLO(trained_path)
    print("✅ 使用訓練後模型")
else:
    model = YOLO("yolov8n.pt")
    print("⚠️ 尚未有有效 labels，使用預訓練模型")

# ---------- 下載測試影片 ----------
video_path = "/content/sample_video.avi"
if not os.path.exists(video_path):
    !wget -O /content/sample_video.avi https://raw.githubusercontent.com/opencv/opencv/master/samples/data/vtest.avi

# ---------- 影片物件偵測 ----------
output_path = f"{PROJECT_DIR}/result_detect.mp4"

cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter(
    output_path,
    cv2.VideoWriter_fourcc(*"mp4v"),
    fps,
    (w, h)
)

for r in model.predict(source=video_path, stream=True):
    frame = r.orig_img.copy()

    if r.boxes is not None:
        for b in r.boxes:
            x1, y1, x2, y2 = map(int, b.xyxy[0])
            cls = int(b.cls)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(
                frame,
                model.names[cls],
                (x1, y1 - 5),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (0, 255, 0),
                2
            )

    out.write(frame)

cap.release()
out.release()

print(f"✅ 偵測完成，輸出影片：{output_path}")

# ---------- 顯示結果影片 ----------
display(HTML(f"""
<video width="640" height="480" controls>
  <source src="{output_path}" type="video/mp4">
</video>
"""))

# ---------- 列出資料夾結構 ----------
print("\n📁 yolo_data 結構：")
for root, dirs, files in os.walk(DATA_DIR):
    level = root.replace(DATA_DIR, '').count(os.sep)
    indent = ' ' * 4 * level
    print(f"{indent}{os.path.basename(root)}/")
    for f in files:
        print(f"{indent}    {f}")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 125.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 12.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incomp

/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:653: RuntimeWarning: Mean of empty slice.
  ax.plot(px, py.mean(1), linewidth=3, color="blue", label=f"all classes {ap[:, 0].mean():.3f} mAP@0.5")
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:697: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:697: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.12/dist-packages/ultraly

                   all          2          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, cannot compute metrics without labels

1 epochs completed in 0.001 hours.
Optimizer stripped from /content/drive/MyDrive/20251226AI/yolo_project/yolo_finetune8/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/MyDrive/20251226AI/yolo_project/yolo_finetune8/weights/best.pt, 6.2MB

Validating /content/drive/MyDrive/20251226AI/yolo_project/yolo_finetune8/weights/best.pt...
Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 36.3it/s 0.0s


/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:653: RuntimeWarning: Mean of empty slice.
  ax.plot(px, py.mean(1), linewidth=3, color="blue", label=f"all classes {ap[:, 0].mean():.3f} mAP@0.5")
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:697: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:697: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.12/dist-packages/ultraly

                   all          2          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, cannot compute metrics without labels
Speed: 0.3ms preprocess, 5.9ms inference, 0.0ms loss, 3.5ms postprocess per image
Results saved to /content/drive/MyDrive/20251226AI/yolo_project/yolo_finetune8
⚠️ 尚未有有效 labels，使用預訓練模型

video 1/1 (frame 1/795) /content/sample_video.avi: 480x640 3 persons, 1 car, 1 truck, 1 bird, 16.6ms
video 1/1 (frame 2/795) /content/sample_video.avi: 480x640 3 persons, 1 car, 1 truck, 6.4ms
video 1/1 (frame 3/795) /content/sample_video.avi: 480x640 3 persons, 1 car, 1 truck, 1 bird, 6.2ms
video 1/1 (frame 4/795) /content/sample_video.avi: 480x640 3 persons, 1 car, 1 truck, 1 bird, 6.2ms
video 1/1 (frame 5/795) /content/sample_video.avi: 480x640 3 persons, 1 car, 1 truck, 1 bird, 6.2ms
video 1/1 (frame 6/795) /content/sample_video.avi: 480x640 3 persons, 1 car, 1 truck, 1 bird, 6.6ms
video 1/1 (frame 7/795) /content/sample_video.avi: 4


📁 yolo_data 結構：
yolo_data/
    data.yaml
    data.yaml.gdoc
    images/
        train/
            people.jpg
            car1.jpg
        val/
            car1.jpg
            people.jpg
    labels/
        train.cache
        val.cache
        train/
            people.txt.gdoc
            car1.txt.gdoc
        val/
            people.txt.gdoc
            car1.txt.gdoc
